In [1]:
!pip install -r requirements.txt

In [2]:
import time
import requests

In [3]:
# Part 1: Scraping Event URLs from the List Pages

import requests
from bs4 import BeautifulSoup
import time

def get_event_urls(page_url):
    # Send a GET request to the given page URL
    response = requests.get(page_url)

    # Check if the request was successful (status code 200)
    if response.status_code != 200:
        print(f"Failed to retrieve page: {page_url}")
        return []
    
    # Parse the response content with BeautifulSoup to navigate the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Define the CSS selector to target the specific elements that contain the event URLs
    selector = "div.search-result-preview > div > h3 > a"

    # Use the selector to find all matching elements (links to event details)
    a_elements = soup.select(selector)

    # Extract the 'href' attribute (URL) from each of these elements
    return [element['href'] for element in a_elements]

def scrape_events(base_url, total_pages):
    all_event_urls = []  # Initialize an empty list to store all event URLs

    # Loop through all pages
    for page in range(1, total_pages + 1):
        # Construct the URL for each page by appending the page number to the base URL
        page_url = f"{base_url}/page/{page}"

        # Call the function to get event URLs from the current page
        event_urls = get_event_urls(page_url)

        # Add the retrieved URLs to the master list
        all_event_urls.extend(event_urls)

        # Sleep for 1 second between requests to avoid overwhelming the server
        time.sleep(1)

    # Return the complete list of event URLs
    return all_event_urls

# Base URL for the 'events' section of the Visit Seattle website
base_url = "https://visitseattle.org/events"

# Total number of pages to scrape - this should be set based on the actual number of pages available
total_pages = 48  # Update this based on the number of pages on the site

# Call the function and store the list of URLs
event_urls = scrape_events(base_url, total_pages)

# Print the list of event URLs
for url in event_urls:
    print(url)



https://visitseattle.org/events/john-cole-peter-de-lory/
https://visitseattle.org/events/the-puppet-masters-making-fools-of-us-all/
https://visitseattle.org/events/whim-whim-winter-24/
https://visitseattle.org/events/compania-nacional-de-danza/
https://visitseattle.org/events/metropolis/
https://visitseattle.org/events/andy-coe-band/
https://visitseattle.org/events/align/
https://visitseattle.org/events/billy-idol-and-duran-duran-tribute-night/
https://visitseattle.org/events/crack-sabbath/
https://visitseattle.org/events/harlem-globetrotters/
https://visitseattle.org/events/inked-up-conversation-with-john-ellis/
https://visitseattle.org/events/jukebox-quartet-arcobaleno-strings/
https://visitseattle.org/events/pike-place-market-local-appreciation-day/
https://visitseattle.org/events/seattle-baroque-orchestra-party-bohemienne/
https://visitseattle.org/events/seattle-thunderbirds-vs-everett-silvertips-2/
https://visitseattle.org/events/the-budos-band/
https://visitseattle.org/events/the

In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import html

# Part 1: Scrape event URLs from the list pages

def get_event_urls(page_url):
    # Send a GET request to the given page URL
    response = requests.get(page_url)

    # Check if the request was successful (status code 200)
    if response.status_code != 200:
        print(f"Failed to retrieve page: {page_url}")
        return []
    
    # Parse the response content with BeautifulSoup to navigate the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Define the CSS selector to target the specific elements that contain the event URLs
    selector = "div.search-result-preview > div > h3 > a"

    # Use the selector to find all matching elements (links to event details)
    a_elements = soup.select(selector)

    # Extract the 'href' attribute (URL) from each of these elements
    return [element['href'] for element in a_elements]

def scrape_events(base_url, total_pages):
    all_event_urls = []  # Initialize an empty list to store all event URLs

    # Loop through all pages
    for page in range(1, total_pages + 1):
        # Construct the URL for each page by appending the page number to the base URL
        page_url = f"{base_url}/page/{page}"

        # Call the function to get event URLs from the current page
        event_urls = get_event_urls(page_url)

        # Add the retrieved URLs to the master list
        all_event_urls.extend(event_urls)

        # Sleep for 1 second between requests to avoid overwhelming the server
        time.sleep(1)

    # Return the complete list of event URLs
    return all_event_urls


# Function to get latitude and longitude from Nominatim
def get_lat_lon(location):
    params = {'q': location, 'format': 'json'}
    response = requests.get('https://nominatim.openstreetmap.org/search', params=params)
    data = response.json()
    if data:
        return data[0]['lat'], data[0]['lon']
    return None, None

# Function to get weather information
def get_weather(lat, lon):
    # Construct URL for National Weather Service API
    points_url = f"https://api.weather.gov/points/{lat},{lon}"
    try:
        points_response = requests.get(points_url)
        points_response.raise_for_status()
        grid_id, grid_x, grid_y = points_response.json()['properties']['gridId'], points_response.json()['properties']['gridX'], points_response.json()['properties']['gridY']

        # Fetching the detailed forecast
        forecast_url = f"https://api.weather.gov/gridpoints/{grid_id}/{grid_x},{grid_y}/forecast"
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        
        # Extracting the detailed forecast information
        forecast_data = forecast_response.json()
        periods = forecast_data['properties']['periods']
        detailed_forecast = periods[0]['detailedForecast'] if periods else "No detailed forecast data"

        return detailed_forecast

    except requests.RequestException as e:
        return f"Weather data not available: {e}"


def clean_text(text):
    text = html.unescape(text)  # Convert HTML entities
    text = text.replace('?', '')  # Remove specific problem characters
    # Additional cleaning logic can be added here
    return text

# Function to scrape event details (including location and weather lookup)
def scrape_event_details(event_url):
    try:
        response = requests.get(event_url)
        response.encoding = 'utf-8'  # Ensure using UTF-8 encoding
        soup = BeautifulSoup(response.text, "html.parser")
    except requests.RequestException as e:
        print(f"Error retrieving event details: {event_url}, Error: {e}")
        return None

    # CSS selectors
    name_selector = 'div.container-event-detail > div:nth-child(1) > div.medium-6.columns.event-top > h1'
    date_selector = 'div.container-event-detail > div:nth-child(1) > div.medium-6.columns.event-top > h4 > span:nth-child(1)'
    location_selector = 'div.container-event-detail > div:nth-child(1) > div.medium-6.columns.event-top > h4 > span:nth-child(2)'
    event_type_selector = 'div.container-event-detail > div:nth-child(1) > div.medium-6.columns.event-top > a:nth-child(3)'
    region_selector = 'div.container-event-detail > div:nth-child(1) > div.medium-6.columns.event-top > a:nth-child(4)'

    # Extract event details
    name = clean_text(soup.select_one(name_selector).get_text(strip=True)) if soup.select_one(name_selector) else 'N/A'
    date = clean_text(soup.select_one(date_selector).get_text(strip=True)) if soup.select_one(date_selector) else 'N/A'
    location = clean_text(soup.select_one(location_selector).get_text(strip=True)) if soup.select_one(location_selector) else 'N/A'
    event_type = clean_text(soup.select_one(event_type_selector).get_text(strip=True)) if soup.select_one(event_type_selector) else 'N/A'
    region = clean_text(soup.select_one(region_selector).get_text(strip=True)) if soup.select_one(region_selector) else 'N/A'

    # Get latitude and longitude for the location
    lat, lon = get_lat_lon(location)

    # Get weather data
    weather = get_weather(lat, lon) if lat and lon else 'Weather data not available'

    # Create a dictionary with all the event details
    event_details = {
        "Name": name,
        "Date": date,
        "Location": location,
        "Type": event_type,
        "Region": region,
        "Latitude": lat,
        "Longitude": lon,
        "Weather": weather
    }

    return event_details

# Main script

base_url = "https://visitseattle.org/events"
total_pages = 48  # Update based on actual page count

event_urls = scrape_events(base_url, total_pages)

# Scrape details for each event URL

events_data = []

for url in event_urls:
    event_details = scrape_event_details(url)
    if event_details is not None:
        events_data.append(event_details)

# Write data to CSV

csv_columns = ["Name", "Date", "Location", "Type", "Region", "Latitude", "Longitude", "Weather"]
csv_file = "events.csv"

try:
    with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in events_data:
            writer.writerow(data)
except IOError as e:
    print(f"I/O error: {e}")

print(f"Data scraping and storage completed. {len(events_data)} events have been processed.")


Data scraping and storage completed. 414 events have been processed.
